### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
%pip install langchain

In [2]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [12]:
%pip install langchain_groq

Note: you may need to restart the kernel to use updated packages.


In [13]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7460795b11c0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7460795b1a30>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
%pip install langchain_core

Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [9]:
result

AIMessage(content='Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, singular)\n* **Salut, comment allez-vous ?** (Informal, plural)\n\n\nThe best choice depends on the level of formality you want to convey and who you are speaking to. \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 21, 'total_tokens': 109, 'completion_time': 0.16, 'prompt_time': 0.00015369, 'queue_time': 0.00359378, 'total_time': 0.16015369}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-7da4151a-1b4c-4d5e-a088-5a57d5025b37-0', usage_metadata={'input_tokens': 21, 'output_tokens': 88, 'total_tokens': 109})

In [15]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a few ways to translate "Hello, how are you?" to French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal)\n* **Coucou, ça va ?** (Very informal) \n\n\nThe best option depends on the context and your relationship with the person you\'re speaking to. \n'

In [18]:
### Using LCEL- chain the components
chain=model|parser
chained_result = chain.invoke(messages)

In [19]:
chained_result

'Here are a couple of ways to say "Hello, how are you?" in French:\n\n**Formal:**\n\n* **Bonjour, comment allez-vous ?** (This is the most polite and formal way to ask.)\n\n**Informal:**\n\n* **Salut, comment vas-tu ?** (Used with friends and family)\n\n\nLet me know if you have any other phrases you\'d like translated!\n'

In [21]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [24]:
result=prompt.invoke({"language":"French","text":"Hello, How are you ?"})

In [25]:
# ChatPromptTemplate will create a prompt output with SystemMessage and HumanMessage
# which in turn, will be input to the model
result.to_messages()

[SystemMessage(content='Translate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, How are you ?', additional_kwargs={}, response_metadata={})]

In [ ]:
##Chaining together components with LCEL
## This line composes three sequential steps into one chain. The pipe operator (“|”) sends the prompt’s output to the model before passing the model’s response to the parser.  

chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello, How are you ?"})

'Here are a couple of ways to say "Hello, How are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, ça va ?** (Informal) \n\n\nLet me know if you\'d like to know other ways to say this!\n'

In [28]:
%pip install streamlit

Note: you may need to restart the kernel to use updated packages.
